# Theano Backend Derivative Calculation Dev Portal 

In [1]:
import os

import keras.backend as K


Using TensorFlow backend.


## 0. Test Data Loading
This section loads the test symmetry function data set, and load the neural network just to test whether the IO part is working propertly. 

In [2]:
import ModelIO

In [3]:
import Utils.DirNav 

In [4]:
project_dir = Utils.DirNav.get_project_dir()
glycine_dir = os.path.join(project_dir, 'model', 'glycine_opt')
print(glycine_dir)

/home/junmian/Applications/YeltsinNet/model/glycine_opt


In [5]:
h_subnet = ModelIO.read_keras_subnet_ind(glycine_dir, 'H-subnet')

In [6]:
# Import Necessary Test Data
Gfunc_data = ModelIO.read_data_from_pickle(glycine_dir, 'Gfunc_data_test.pkl')

In [7]:
energy_arr = h_subnet.predict(Gfunc_data['H'][3])

In [8]:
energy_arr.shape

(1001, 1)

#### Result: 
Thus, we have proven that Theano backend is capable of loading the weight from tensorflow training set. 

## 1. Copy and Paste Derivative Functions

In [9]:
Gfunc_data['H'][3].shape

(1001, 276)

### Problem: 
The Neural Network is designed to train an ensemble of the same molecule, rather than for individual molecules. Therefore, Theano indicates that we cannot calculate the derivative for many molecules at once. 
<br/>

### Trick:
The only place that we are going to use derivative is (for molecular dynamics simulation), for which we are only going to use derivatives for individual molecules. (Maybe it is also required for the calculations of derivative for the ensemble, which may need another equation, but not for now) The temporary solution is to directly assume that structure of the derivative, and then calculate it. 
<br/>

### Assumption: 
Gfunc_data_ind = (1, 1, n_symm_func) <br/>
energy_predict = (1, 1)   <br/>

_Therefore_ <br/>
energy = energy_predict[0][0]


In [10]:
def get_subnet_deriv_tf(subnet, n_symm_func):
    """Generate a dE/dG function for a given elemental sub-neural network.

        Args:
            subnet: keras.model object for the sub-neural network.
            nb_symm: int, number of symmetry functions for the vector.

        Outputs:
            subnet_deriv([G_ind]), a function that takes the symmetry vector of shape (1, nb_symm) of the given atom
            and return the dE/dG matrix for the atom.

    """
    G_tensor_ind = K.placeholder(shape = (1, n_symm_func))
    E_tensor_ind = subnet(G_tensor_ind)
    subnet_deriv = K.function([G_tensor_ind], K.gradients(E_tensor_ind, [G_tensor_ind]))

    return subnet_deriv

In [11]:
n_symm_func = 276

h_subnet_deriv = get_subnet_deriv_tf(h_subnet, n_symm_func)

In [12]:
deriv_output_tf = h_subnet_deriv([Gfunc_data['H'][3][0:1,:]])

In [13]:
print(deriv_output_tf)

[array([[ 4.60946467e-04,  1.02542213e-03, -8.05303548e-03,
        -2.26611714e-03,  1.66188576e-04,  1.77293597e-03,
         4.17877221e-04,  5.10764075e-04,  2.84282770e-03,
         2.48019607e-03,  1.94754265e-03, -2.12259576e-04,
         8.74885591e-04,  3.42079671e-04,  1.55409143e-04,
         1.40619278e-03,  1.27050187e-03,  2.81268614e-04,
         6.33427349e-04,  1.93949975e-03,  1.40992971e-03,
         7.77234789e-04,  1.63180160e-03,  3.96909018e-04,
        -5.32109989e-04,  5.64240327e-06,  4.50871419e-03,
         1.41532707e-03, -1.15749310e-03, -6.65612752e-04,
         2.56625121e-03,  6.70008885e-05,  3.41043429e-04,
        -1.02040730e-03, -1.34436949e-03, -2.23028357e-03,
        -2.09460035e-03, -1.21425698e-03, -3.70808353e-04,
        -3.25866218e-04, -2.96630315e-03, -2.49305204e-03,
        -1.27795129e-03,  3.63370869e-04, -1.06673595e-03,
        -4.40049276e-04, -1.38552394e-03,  3.48777277e-04,
         4.86407895e-04,  3.12436675e-03,  6.97570527e-

In [26]:
deriv_output_th[0].shape

(1, 276)

In [15]:

def get_subnet_deriv_arr_theano(subnet_list, nb_symm):
    """Generated the array for the dE/dG function.
    !! For Elemental Neural Network only

        Args:
            subnet_list: a list that includes all the sub-neural network with its element
            nb_symm: number of symmetry functions for the vector.

        Outputs:
            a pandas.Serires() array of subnet_deriv([G_ind]), a function that takes the symmetry
            vector of shape (1, nb_symm) of the given atom
            and return the dE/dG matrix for the atom.
    """
    subnet_deriv_arr = {}
    for element, subnet in subnet_list.items():
        subnet_deriv_arr[element] = get_subnet_deriv_theano(subnet, nb_symm)

    return subnet_deriv_arr





In [ ]:
def calculate_subnet_deriv_value(subnet_deriv_arr, element, Gfunc_ind):
    """

            Args:
                Gfunc_ind: Gfunc for individual atoms in the shape
                            (1, symm_func)

            Outputs:
                output: dE(subnet)/dG for the given atom (determined by Gfunc_ind)
                (1,symm_func)


    Assumes that Gfunc_ind has the same No. Symmetry function as the function for dE_dG_arr
    """
    subnet_deriv = subnet_deriv_arr[element]
    output =  np.array(subnet_deriv([Gfunc_ind])).reshape(Gfunc_ind.shape).astype(np.float64)
    return output


## 1. Prototype Theano Functions 